# One-Photon Calcium Imaging Example

This notebook demonstrates how to use the ndx-microscopy extension for storing one-photon calcium imaging data in NWB format.

## Import Required Libraries

Import necessary packages from NWB, ndx-microscopy, and ndx-ophys-devices extensions, along with other utility libraries.

In [ ]:
from datetime import datetime
from uuid import uuid4
import matplotlib.pyplot as plt
import numpy as np
from pynwb import NWBFile, NWBHDF5IO

from ndx_microscopy import (
    Microscope, 
    PlaneAcquisition,
    ExcitationLightPath,
    EmissionLightPath,
    PlanarImagingSpace,
    PlanarMicroscopySeries,
    Segmentation2D,
    SummaryImage,
    MicroscopyResponseSeries,
    MicroscopyResponseSeriesContainer
)

from ndx_ophys_devices import ExcitationSource, BandOpticalFilter, DichroicMirror, Photodetector, Indicator

## Create NWB File

Initialize a new NWB file with basic session information.

In [ ]:
nwbfile = NWBFile(
    session_description="One-photon calcium imaging session",
    identifier=str(uuid4()),
    session_start_time=datetime.now(),
    lab="Neural Imaging Lab",
    institution="University of Neuroscience",
    experiment_description="Gfp imaging in Hippocampus",
)

## Set Up Microscope and Components

Configure the microscope and its various components including:
- Main microscope device
- Blue LED excitation source (470nm for GFP)
- Excitation filter (470/40nm bandpass)
- Dichroic mirror (495nm cutoff)
- Emission filter (525/50nm bandpass)
- CMOS camera detector
- GFP indicator

In [ ]:
# Set up microscope
microscope = Microscope(
    name='1p-scope',
    description='Custom one-photon microscope for calcium imaging',
    manufacturer='Custom Build',
    model='1P-Special',
    technique='widefield',
)
nwbfile.add_device(microscope)

# Configure LED light source
laser = ExcitationSource(
    name="blue_laser",
    illumination_type="LED",
    manufacturer="Thorlabs",
    model="M470L4",
    excitation_mode="one-photon",
    excitation_wavelength_in_nm=470.0,  # Blue LED optimal for GFP excitation
    power_in_W=0.18,  # 180mW typical for this LED
    intensity_in_W_per_m2=1500.0,  # ~1500 W/m² at sample
    exposure_time_in_s=0.001,  # 1ms exposure time
)
nwbfile.add_device(laser)

In [ ]:
# Configure optical components
excitation_filter = BandOpticalFilter(
    name="excitation_filter",
    filter_type="Bandpass",
    manufacturer="Chroma",
    model="ET470/40x",
    center_wavelength_in_nm=470.0,
    bandwidth_in_nm=40.0,
)
nwbfile.add_device(excitation_filter)

dichroic = DichroicMirror(
    name="primary_dichroic",
    manufacturer="Chroma",
    model="T495lpxr",  # Common dichroic for GFP imaging
    cut_on_wavelength_in_nm=495.0,  # Transmits >495nm
    cut_off_wavelength_in_nm=490.0,  # Reflects <490nm
    transmission_band_in_nm=[495.0, 750.0],  # Transmits emission light
    reflection_band_in_nm=(400.0, 490.0),  # Reflects excitation light
    angle_of_incidence_in_degrees=45.0,
)
nwbfile.add_device(dichroic)

emission_filter = BandOpticalFilter(
    name="emission_filter",
    filter_type="Bandpass",
    manufacturer="Chroma",
    model="ET525/50m",
    center_wavelength_in_nm=525.0,
    bandwidth_in_nm=50.0,
)
nwbfile.add_device(emission_filter)

detector = Photodetector(
    name="camera",
    detector_type="CMOS",
    manufacturer="Hamamatsu",
    model="ORCA-Flash4.0",
    detected_wavelength_in_nm=525.0,
)
nwbfile.add_device(detector)

In [ ]:
# Create indicator
indicator = Indicator(
    name="gfp",
    label="GFP",
    description="Green fluorescent protein for one-photon imaging",
    manufacturer="Addgene",
    injection_brain_region="Hippocampus",
    injection_coordinates_in_mm=[-1.8, 2.0, 1.2],
)

## Configure Light Paths

Set up the excitation (blue LED) and emission (GFP) light paths, connecting the previously defined components.

In [ ]:
# Configure excitation path
excitation = ExcitationLightPath(
    name="1p_excitation",
    description="Blue LED excitation pathway",
    excitation_source=laser,
    excitation_filter=excitation_filter,
    dichroic_mirror=dichroic,
)
nwbfile.add_lab_meta_data(excitation)

# Configure emission path
emission = EmissionLightPath(
    name="gfp_emission",
    description="GFP emission pathway",
    indicator=indicator,
    photodetector=detector,
    emission_filter=emission_filter,
    dichroic_mirror=dichroic,
)
nwbfile.add_lab_meta_data(emission)

## Define Imaging Space and Create Example Data

Set up the imaging space parameters for hippocampal imaging and create sample imaging data.

In [ ]:
# Define imaging space
illumination_pattern = PlaneAcquisition(
    name="plane_acquisition",
    description="Widefield fluorescence imaging",
    plane_thickness_in_um=5.0,
)
imaging_space = PlanarImagingSpace(
    name="hippo_plane1",
    description="CA1 region of hippocampus",
    pixel_size_in_um=[1.0, 1.0],
    origin_coordinates=[-1.8, 2.0, 1.2],  # Matches the indicator injection coordinates
    location="Hippocampus, CA1 region",
    reference_frame="bregma",
    orientation="RAS",  # Right-Anterior-Superior
    illumination_pattern=illumination_pattern,
)

# Create example imaging data
frames = 1000
height = 512
width = 512
data = np.random.rand(frames, height, width)

# Create imaging series
imaging_series = PlanarMicroscopySeries(
    name='imaging_data',
    description='One-photon calcium imaging',
    microscope=microscope,
    excitation_light_path=excitation,
    emission_light_path=emission,
    planar_imaging_space=imaging_space,
    data=data,
    unit='a.u.',
    rate=30.0,
    starting_time=0.0
)
nwbfile.add_acquisition(imaging_series)

## ROI Segmentation and Analysis

Create and analyze regions of interest (ROIs) in the imaging data:

In [ ]:
# Create ophys processing module
ophys_module = nwbfile.create_processing_module(
    name='ophys',
    description='Optical physiology processing module'
)

# Create summary images
mean_image = SummaryImage(
    name='mean',
    description='Mean intensity projection',
    data=np.mean(data, axis=0)
)

max_image = SummaryImage(
    name='max',
    description='Maximum intensity projection',
    data=np.max(data, axis=0)
)

# Create segmentation
segmentation = Segmentation2D(
    name='rois',
    description='Manual ROI segmentation',
    planar_imaging_space=imaging_space,
    summary_images=[mean_image, max_image]
)

In [ ]:
# Add ROIs using image masks
roi_mask = np.zeros((height, width), dtype=bool)
roi_mask[256:266, 256:266] = True  # 10x10 ROI
segmentation.add_roi(image_mask=roi_mask)

# Create ROI responses
roi_region = segmentation.create_roi_table_region(
    description='All ROIs',
    region=list(range(len(segmentation.id)))
)

# Extract responses (example calculation)
num_rois = len(segmentation.id)
responses = np.zeros((frames, num_rois))

for i, roi_mask in enumerate(segmentation.image_mask[:]):
    roi_data = data[:, roi_mask]
    responses[:, i] = np.mean(roi_data, axis=1)

# Create response series
response_series = MicroscopyResponseSeries(
    name='roi_responses',
    description='Fluorescence responses from ROIs',
    data=responses,
    rois=roi_region,
    unit='n.a.',
    rate=30.0,
    starting_time=0.0
)

# Create container for response series
response_container = MicroscopyResponseSeriesContainer(
    name='responses',
    microscopy_response_series=[response_series]
)

# Add segmentation and responses to ophys module
ophys_module.add(segmentation)
ophys_module.add(response_container)

## Save and Load NWB File

Write the NWB file to disk and demonstrate how to read it back.

In [ ]:
# Save file
with NWBHDF5IO('one_photon_calcium_imaging.nwb', 'w') as io:
    io.write(nwbfile)

# Read file and access data
with NWBHDF5IO("one_photon_calcium_imaging.nwb", "r") as io:
    nwbfile = io.read()

    # Access imaging data
    imaging = nwbfile.acquisition['imaging_data']
    raw_data = imaging.data[:]

    # Access ROI data
    ophys = nwbfile.processing['ophys']
    rois = ophys['rois']
    roi_masks = rois.image_mask[:]

    # Access responses
    responses = ophys['responses']
    roi_data = responses['roi_responses'].data[:]

## Data Visualization

Create plots to visualize the imaging data, ROI masks, and fluorescence responses.

In [ ]:
# Create a figure with 2 rows and 2 columns
fig = plt.figure(figsize=(20, 15))

# First frame
ax1 = plt.subplot(2, 2, 1)
im1 = ax1.imshow(raw_data[0], cmap="gray")
ax1.set_title("First Frame")
plt.colorbar(im1, ax=ax1)

# ROI masks
ax2 = plt.subplot(2, 2, 2)
for i, mask in enumerate(roi_masks):
    im2 = ax2.imshow(mask, alpha=0.5, cmap="viridis")
    ax2.set_title(f"ROI {i+1} Mask")
plt.colorbar(im2, ax=ax2)

# ROI responses over time
ax3 = plt.subplot(2, 1, 2)
ax3.plot(roi_data)
ax3.set_title('ROI Fluorescence Response')
ax3.set_xlabel('Frame Number')
ax3.set_ylabel('Fluorescence (a.u.)')
ax3.grid(True)

plt.tight_layout()
plt.show()